In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
from sklearn import svm
from scipy.stats.stats import pearsonr
from sklearn.preprocessing import MinMaxScaler
import seaborn as sn
import matplotlib.font_manager
from sklearn.metrics import confusion_matrix
import itertools
%matplotlib inline

In [2]:
DF = pd.read_csv("interviewData.csv")
print("total:", DF.shape)
DF.head()

total: (1440, 400)


,A_1,A_2,A_3,A_4,A_5,A_6,A_7,A_8,A_9,A_10,...,diex_WS1,diey_WS1,ecid,site_WS1,totaldietestseconds_WS1,touchdownseq_WS1,wafername_WS1,hardbin_FT1,site_FT1,totaldietestseconds_FT1
0,1,0,0,0,1,1,0,0,0,0,...,-3,15,4.05E+10,1,76245,69,T27H82-07G0,1,3,77054
1,-2,1,2,3,2,2,2,0,0,1,...,2,1,0404E95646B4,3,75269,119,T27H82-07G0,1,8,75299
2,0,0,2,0,0,0,0,0,-1,0,...,2,23,0404730E93DF,5,78917,35,T27H82-07G0,1,4,77512
3,0,0,0,0,1,0,0,0,0,0,...,17,14,0404A52D1C79,2,83262,79,T27H82-07G0,1,1,82429
4,-2,1,-1,0,1,1,1,0,0,0,...,6,27,0404FF90DCE8,5,73490,19,T27H82-07G0,1,6,75694


In [3]:
features_ls = list(DF.keys()) ## exclude ecid, wafername_WS1, and hardbin_FT1
#non_features = ['ecid', 'wafername_WS1', 'hardbin_FT1']
non_features = ['ecid', 'wafername_WS1', 'hardbin_FT1', 'diex_WS1', 'diey_WS1', 'site_WS1', 'site_FT1']
for non_f in non_features:
    features_ls.remove(non_f)

In [4]:
features = DF[features_ls]
features.head()

,A_1,A_2,A_3,A_4,A_5,A_6,A_7,A_8,A_9,A_10,...,J_52,J_53,J_54,J_55,J_56,J_57,J_58,totaldietestseconds_WS1,touchdownseq_WS1,totaldietestseconds_FT1
0,1,0,0,0,1,1,0,0,0,0,...,0.065226,0.497933,0.624575,0.367481,0.620765,1.545064,1.384001,76245,69,77054
1,-2,1,2,3,2,2,2,0,0,1,...,0.761168,0.586969,0.022726,0.458222,0.412771,1.558442,1.219390,75269,119,75299
2,0,0,2,0,0,0,0,0,-1,0,...,0.531965,0.457407,0.618359,0.916580,0.111832,1.538462,1.300521,78917,35,77512
3,0,0,0,0,1,0,0,0,0,0,...,0.061631,0.151994,0.234272,0.061631,1.010847,2.320926,1.558374,83262,79,82429
4,-2,1,-1,0,1,1,1,0,0,0,...,0.059124,0.704903,0.059124,0.027381,0.327142,1.931330,0.951548,73490,19,75694


In [5]:
scaler = MinMaxScaler()

In [11]:
import warnings
warnings.filterwarnings('ignore')

In [12]:
feature_list = list(features.keys())
features[feature_list] = scaler.fit_transform(features[feature_list])

In [13]:
features.head()

,A_1,A_2,A_3,A_4,A_5,A_6,A_7,A_8,A_9,A_10,...,J_52,J_53,J_54,J_55,J_56,J_57,J_58,totaldietestseconds_WS1,touchdownseq_WS1,totaldietestseconds_FT1
0,0.444444,0.571429,0.428571,0.428571,0.500,0.666667,0.500000,0.625,0.714286,0.666667,...,0.035324,0.271997,0.327525,0.178024,0.369854,0.335475,0.357335,0.158679,0.575000,0.128300
1,0.111111,0.714286,0.714286,0.857143,0.625,0.777778,0.833333,0.625,0.714286,0.833333,...,0.413425,0.320686,0.011694,0.222003,0.245913,0.341259,0.281740,0.152071,0.991667,0.109506
2,0.333333,0.571429,0.714286,0.428571,0.375,0.555556,0.500000,0.625,0.571429,0.666667,...,0.288901,0.249836,0.324263,0.444154,0.066586,0.332620,0.318998,0.176771,0.291667,0.133204
3,0.333333,0.571429,0.428571,0.428571,0.500,0.555556,0.500000,0.625,0.714286,0.666667,...,0.033371,0.082826,0.122707,0.029788,0.602300,0.670961,0.437413,0.206191,0.658333,0.185858
4,0.111111,0.714286,0.285714,0.428571,0.500,0.666667,0.666667,0.625,0.714286,0.666667,...,0.032009,0.385176,0.030795,0.013188,0.194888,0.502498,0.158738,0.140025,0.158333,0.113736


In [14]:
## The class we want to train
Pass_data_x = []
Pass_data_y = []

## Outlier
Fail_data_x = []
Fail_data_y = []

for k in range(len(DF)):
    if DF['hardbin_FT1'][k] == 1:
        feature_tmp = []
        for s in features.keys():
            feature_tmp.append(features[s][k])
        Pass_data_x.append(feature_tmp)
        Pass_data_y.append(DF['hardbin_FT1'][k])
    else:
        feature_tmp = []
        for s in features.keys():
            feature_tmp.append(features[s][k])
        Fail_data_x.append(feature_tmp)
        Fail_data_y.append(DF['hardbin_FT1'][k])

Pass_data_x = np.array(Pass_data_x)
Pass_data_y = np.array(Pass_data_y)
Fail_data_x = np.array(Fail_data_x)
Fail_data_y = np.array(Fail_data_y)

In [16]:
print("Pass:", len(Pass_data_y))
print("Fail:", len(Fail_data_y))

Pass: 1429
Fail: 11


In [17]:
def splitData(Data_x, Data_y, ratio):
    num = int(round(len(Data_x)*ratio))
    par = len(Data_x) - num
    x_main = Data_x[:num]
    y_main = Data_y[:num]
    x_mini = Data_x[-par:]
    y_mini = Data_y[-par:]
    return x_main, y_main, x_mini, y_mini

In [18]:
def unison_shuffled_copies(a, b):
    assert len(a) == len(b)
    p = np.random.permutation(len(a))
    return a[p], b[p]

In [19]:
Pass_data_x, Pass_data_y = unison_shuffled_copies(Pass_data_x, Pass_data_y)

In [20]:
X_train, Y_train, X_test, Y_test = splitData(Pass_data_x, Pass_data_y, 0.8)
X_outlier = Fail_data_x
Y_outlier = Fail_data_y
Y_outlier_re = np.array([-1 for i in range(len(Y_outlier))])

In [22]:
print("Train:", len(X_train))
print("Test:", len(X_test))
print("Outlier:", len(X_outlier))

Train: 1143
Test: 286
Outlier: 11


## Isolation Forest

In [23]:
from sklearn.ensemble import IsolationForest

In [25]:
rng = np.random.RandomState(42)
iso_clf = IsolationForest(max_samples=100, random_state=rng)

In [27]:
iso_clf.fit(X_train)
y_pred_train = iso_clf.predict(X_train)
y_pred_test = iso_clf.predict(X_test)
y_pred_outliers = iso_clf.predict(X_outlier)